# Preparação de dados

## Importação de módulos

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from IPython.display import HTML
import urllib
import bs4 #this is beautiful soup
import re # regular expressions

ModuleNotFoundError: No module named 'matplotlib'

## Leitura dos dados

In [57]:
dados = pd.read_csv('dengue_capitais.csv.gz')

In [58]:
dados.head()

,dt_notificacao,co_cid,co_municipio_notificacao,co_municipio_residencia,dt_diagnostico_sintoma,dt_digitacao
0,2012-12-27 00:00:00,A90,110020,110020,2012-12-21 00:00:00,2013-02-22 00:00:00
1,2013-01-14 00:00:00,A90,110020,110020,2013-01-01 00:00:00,2013-02-22 00:00:00
2,2013-02-12 00:00:00,A90,500270,500270,2013-02-09 00:00:00,2013-02-23 00:00:00
3,2013-02-08 00:00:00,A90,520870,520870,2013-02-01 00:00:00,2013-03-01 00:00:00
4,2013-02-27 00:00:00,A90,431490,431490,2013-02-26 00:00:00,2013-02-27 00:00:00


In [59]:
dados.dtypes

dt_notificacao              object
co_cid                      object
co_municipio_notificacao     int64
co_municipio_residencia      int64
dt_diagnostico_sintoma      object
dt_digitacao                object
dtype: object

In [121]:
dados.isnull().sum()

dt_notificacao                 0
co_cid                         0
co_municipio_notificacao       0
co_municipio_residencia        0
dt_diagnostico_sintoma         8
dt_digitacao                8335
dtype: int64

### Problema:
O que fazer com os dados omitidos? Usar a data de notificação?

In [96]:
municipios = sorted(dados.co_municipio_notificacao.unique().tolist())
residencias = sorted(dados.co_municipio_residencia.unique().tolist())

print(type(municipios), len(municipios), type(residencias), len(residencias))

<class 'list'> 2055 <class 'list'> 28


### Problema: 
Temos 2055 municípios de notificação e apenas 28 de residência

In [81]:
data_notificacao = pd.to_datetime(dados.dt_notificacao, format = '%Y-%m-%d %H:%M:%S')
data_diagnostico = pd.to_datetime(dados.dt_diagnostico_sintoma, format = '%Y-%m-%d %H:%M:%S')
data_digitacao = pd.to_datetime(dados.dt_digitacao, format = '%Y-%m-%d %H:%M:%S')

In [98]:
ano_notificacao = data_notificacao.dt.year
semana_notificacao = data_notificacao.dt.week

In [99]:
ano_diagnostico = data_diagnostico.dt.year
semana_diagnostico = data_diagnostico.dt.week

In [100]:
ano_digitacao = data_digitacao.dt.year
semana_digitacao = data_digitacao.dt.week

In [108]:
dengue = {'ano_notificacao': ano_notificacao, 
          'semana_notificacao': semana_notificacao,
          'ano_diagnostico': ano_digitacao,
          'semana_diagnostico': semana_diagnostico,
          'ano_digitacao': ano_digitacao,
          'semana_digitacao': semana_digitacao,
          'municipio': dados.co_municipio_notificacao,
          'residencia': dados.co_municipio_residencia}

In [119]:
ano_diagnostico.unique()

array([2012., 2013., 2007., 2006., 2014., 2015., 1983., 2018., 2019.,
       1988., 2001., 2016., 2017., 2004., 2009., 2008., 2005., 2010.,
       2011., 1941., 1987., 1974., 1971., 1929., 1950., 2000., 1964.,
       2003., 1981., 1966., 1999., 1980., 1977., 1952., 1985., 1978.,
       1997., 1956., 1990., 1992., 1945., 1975., 1994., 1996., 1993.,
       1986., 1989., 1967., 2002., 1991., 1955., 1972., 1973., 1961.,
       1979., 1984., 1940., 1960., 1969., 1934., 1995., 1954., 1976.,
       1968., 1970., 1982., 1947., 1998., 1944., 1949., 1951., 1936.,
       1958., 1965., 1927., 1963., 1962., 1959., 1957., 1938., 1946.,
       1948., 1953., 1943., 1937., 1931.,   nan, 1942., 1939., 1935.,
       1932., 1926., 1915., 1922., 1911., 1933., 1923., 1921.])

In [115]:
semana_diagnostico.unique()

array([51.,  1.,  6.,  5.,  9.,  3.,  4.,  8.,  7., 40.,  2., 12., 10.,
       42., 11., 23., 33., 35., 19., 31., 32., 21., 15., 18., 43., 45.,
       29., 14., 44., 41., 48., 47., 22., 49., 27., 46., 50., 13., 28.,
       16., 17., 38., 34., 36., 37., 20., 52., 53., 24., 39., 30., 26.,
       25., nan])

In [116]:
ano_digitacao.unique()

array([2013., 2007., 2014., 2015., 2018., 2019., 2012., 2016., 2017.,
         nan, 2009., 2008., 2010., 2011., 2025., 2024., 2020., 2021.])

In [109]:
dengue = pd.DataFrame(dengue)
dengue.head()

,ano_notificacao,semana_notificacao,ano_diagnostico,semana_diagnostico,ano_digitacao,semana_digitacao,municipio,residencia
0,2012,52,2013.0,51.0,2013.0,8.0,110020,110020
1,2013,3,2013.0,1.0,2013.0,8.0,110020,110020
2,2013,7,2013.0,6.0,2013.0,8.0,500270,500270
3,2013,6,2013.0,5.0,2013.0,9.0,520870,520870
4,2013,9,2013.0,9.0,2013.0,9.0,431490,431490


In [110]:
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3684688 entries, 0 to 3684687
Data columns (total 8 columns):
ano_notificacao       int64
semana_notificacao    int64
ano_diagnostico       float64
semana_diagnostico    float64
ano_digitacao         float64
semana_digitacao      float64
municipio             int64
residencia            int64
dtypes: float64(4), int64(4)
memory usage: 224.9 MB
